In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import numpy as np
from keras.preprocessing import image
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
import tensorflow as tf
# Check TensorFlow version
print("TensorFlow Version:", tf.__version__)

TensorFlow Version: 2.9.1


In [3]:
# Define dataset paths (CHANGE THESE TO YOUR LOCAL FOLDERS)
train_path = 'D:/Main_dataset_client/Train'
test_path = 'D:/Main_dataset_client/Test'

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,           # Increased from 40 to 45 degrees
    width_shift_range=0.3,       # Increased from 0.2 to 0.3
    height_shift_range=0.3,      # Increased from 0.2 to 0.3
    shear_range=0.3,             # Increased from 0.2 to 0.3
    zoom_range=0.3,              # Increased from 0.2 to 0.3
    horizontal_flip=True,
    vertical_flip=True,          # Added vertical flipping
    fill_mode='nearest',
    brightness_range=[0.5, 1.5], # Expanded brightness range
    channel_shift_range=50.0   # Added color channel shifts]    # Added contrast variation
)

training_set = train_datagen.flow_from_directory(
    train_path,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

# Test generator remains unchanged (no augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory(
    test_path,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 82 images belonging to 7 classes.
Found 14 images belonging to 7 classes.


In [6]:
# Load Pre-trained Xception model
base_model = Xception(
    input_shape=(299, 299, 3),
    include_top=False,
    weights='imagenet'
)

In [7]:
# Freeze base model
base_model.trainable = False

In [8]:
# Custom classifier
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    Dropout(0.6),
    Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    Dropout(0.5),
    Dense(training_set.num_classes, activation='softmax')
])

In [9]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [10]:
# Callbacks
checkpoint = ModelCheckpoint(
    'best_xception_new.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=1e-7
)

In [11]:
# Train the model
history = model.fit(
    training_set,
    validation_data=test_set,
    epochs=50,
    callbacks=[checkpoint, early_stop, reduce_lr]
)

Epoch 1/50
3/3 [==============================] - 33s 12s/step - loss: 4.9990 - accuracy: 0.2439 - val_loss: 3.9603 - val_accuracy: 0.2143 - lr: 1.0000e-04
Epoch 2/50
3/3 [==============================] - 25s 8s/step - loss: 4.9481 - accuracy: 0.1585 - val_loss: 3.8952 - val_accuracy: 0.2857 - lr: 1.0000e-04
Epoch 3/50
3/3 [==============================] - 26s 10s/step - loss: 4.6736 - accuracy: 0.2317 - val_loss: 3.8370 - val_accuracy: 0.5000 - lr: 1.0000e-04
Epoch 4/50
3/3 [==============================] - 25s 8s/step - loss: 4.4459 - accuracy: 0.3171 - val_loss: 3.7795 - val_accuracy: 0.5714 - lr: 1.0000e-04
Epoch 5/50
3/3 [==============================] - 25s 10s/step - loss: 4.0747 - accuracy: 0.3659 - val_loss: 3.7259 - val_accuracy: 0.7143 - lr: 1.0000e-04
Epoch 6/50
3/3 [==============================] - 25s 10s/step - loss: 3.4903 - accuracy: 0.5000 - val_loss: 3.6736 - val_accuracy: 0.7143 - lr: 1.0000e-04
Epoch 7/50
3/3 [==============================] - 26s 8s/step - lo

In [12]:
# Evaluate the model
eval_results = model.evaluate(test_set)
print(f"Test Loss: {eval_results[0]:.4f} | Test Accuracy: {eval_results[1]:.4f}")

1/1 [==============================] - 4s 4s/step - loss: 2.4397 - accuracy: 1.0000
Test Loss: 2.4397 | Test Accuracy: 1.0000


In [13]:
model.save('D:/Model_Main/Xception_net_client.h5')

In [14]:
# Fine-tuning: Unfreeze some layers
base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False

In [15]:
# Recompile with lower learning rate
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [16]:
# Fine-tuning training
history_fine = model.fit(
    training_set,
    validation_data=test_set,
    epochs=20,
    initial_epoch=history.epoch[-1],
    callbacks=[checkpoint, early_stop, reduce_lr]
)

In [17]:
# Evaluate after fine-tuning
eval_results = model.evaluate(test_set)
print(f"Test Loss: {eval_results[0]:.4f} | Test Accuracy: {eval_results[1]:.4f}")

1/1 [==============================] - 5s 5s/step - loss: 2.4397 - accuracy: 1.0000
Test Loss: 2.4397 | Test Accuracy: 1.0000


In [18]:
model.save('D:/Model_Main/Xception_net_client_tunned.h5')

In [28]:
# Define class labels manually
from tensorflow.keras.utils import load_img, img_to_array

class_labels = {
    0: 'Birds',
    1: 'Butterflies',
    2: 'Frog',
    3: 'Snakes',
    4: 'Odonata',
    5: 'Moths',
    6: 'Lizard'
}

# Load the trained model
from tensorflow.keras.models import load_model
import numpy as np
from keras.preprocessing import image

model = load_model('D:/Model_Main/Xception_net_client.h5')  # Update with your saved model path

# Load and preprocess the image
image_path = 'D:/Less_data_test/c.JPG'  # Update with your image path
# Load and preprocess the image
test_image = load_img(image_path, target_size=(299, 299))  # Corrected function
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image = test_image / 255.0  # Normalize the image


# Predict the class
result = model.predict(test_image)
predicted_class = np.argmax(result)

# Get the predicted label
prediction = class_labels[predicted_class]

print(f'The predicted class is: {prediction}')

1/1 [==============================] - 1s 913ms/step
The predicted class is: Frog
